In [1]:
import pandas as pd
import os
import sqlite3

Preparing Data and Loading the data into Pandas DataFrame

In [2]:
def DataFileSystem(f):
    colnames = ['Header','FileType','CompanyName','DateCreated','TimeCreated','FileName']     
    d = pd.read_csv('gazprom_sample_data/'+filename,names=colnames,header=None,nrows=1)
    return d

def DataMeter(f):
    colnames = ['Header','MeterNumber','MeasurementDate','MeasurementTime','Consumption','NaN']     
    data = pd.read_csv('gazprom_sample_data/'+filename,names=colnames,header=None)
    return data

Function for Creating and Connecting to the DataBase


In [3]:
def ConnectDatabase():
    conn = None
    try:
        conn = sqlite3.connect('Gazprom_db')
    except Error as e:
        print(e)
    return conn    

Main Program

In [4]:
conn = ConnectDatabase()

if conn is not None:

        # create FileSystem table
        try:
            c = conn.cursor()
            c.execute("""CREATE TABLE IF NOT EXISTS FileSystem 
                                (   FileName PRIMARY KEY,
                                    FileType text NOT NULL,
                                    CompanyName text NOT NULL,
                                    DateCreated integer NOT NULL,
                                    TimeCreated integer
                                )""")
        except Error as e:
            print("Error! cannot create the database connection.")
        
        # create MeterData table
        try:
            c = conn.cursor()
            c.execute("""CREATE TABLE IF NOT EXISTS MeterData 
                                (   
                                    MeterNumber text NOT NULL,
                                    MeasurementDate integer NOT NULL,
                                    MeasurementTime integer,
                                    Consumption real ,
                                    FileName text 
                                )""")
        except Error as e:
            print("Error! cannot create the database connection.")    

In [5]:
for filename in os.listdir('gazprom_sample_data'):
   
   if filename.endswith(".SMRT"):
    
    d = DataFileSystem(filename)
    
    if d['Header'][0] != 'HEADR' and d.iloc[-1, d.columns.get_loc("Header")] !='TRAIL':
        print('Error:Wrong format')
    else:    
        del d['Header']
        c.execute("SELECT * FROM FileSystem WHERE FileName = ?", (d['FileName'][0],))
        data=c.fetchall()
        if len(data)==0:
            cols = d.columns.tolist()    
            cols = cols[-1:] + cols[:-1]
            d = d[cols]
            d.to_sql('FileSystem', conn, if_exists='append', index = False)
    

In [6]:
for filename in os.listdir('gazprom_sample_data'):
    
    if filename.endswith(".SMRT"):
        
        data = DataMeter(filename)
        
        #print(data)
        
        if data['Header'][0] != 'HEADR' and data.iloc[-1, data.columns.get_loc("Header")] !='TRAIL':
            print('Error:Wrong format')
        else:
            data = data.iloc[:-1 , :]
            data = data.iloc[1: , :]
            data['MeasurementTime'] = data['MeasurementTime'].astype(int)
            del data['Header']
            del data['NaN']
            f = os.path.splitext(filename)[0]
            data['Filename'] = f
            if conn is not None:

                 # create DummyData table
                try:
                    c = conn.cursor()
                    c.execute("""CREATE TABLE IF NOT EXISTS DummyData 
                                (  
                                    MeterNumber text NOT NULL,
                                    MeasurementDate integer NOT NULL,
                                    MeasurementTime integer,
                                    Consumption real  ,
                                    FileName text 
                                 )""")
                except Error as e:
                     print("Error! cannot create the database connection.")
            
                data.to_sql('DummyData', conn, if_exists='append', index = False)
            
                c.execute('''SELECT COUNT(*) from MeterData  ''')
            
                result=c.fetchall()
            
                if result[0][0] == 0 :
            
                    c.execute("""INSERT INTO MeterData 
                                 SELECT * FROM DummyData
                                 ORDER BY MeterNumber ASC;""")
            
                    c.execute("""DROP TABLE DummyData;""") 
                else :
                    c.execute("""INSERT INTO MeterData
                                 SELECT * 
                                 FROM DummyData 
                                 WHERE NOT EXISTS 
                                 (SELECT * 
                                  FROM MeterData 
                                  WHERE MeterData.MeterNumber = DummyData.MeterNumber 
                                  AND MeasurementDate = MeterData.MeasurementDate 
                                  AND MeasurementTime = MeterData.MeasurementTime
                                 )
                                 ORDER BY MeterNumber ASC;""") 
                  
                    c.execute("""INSERT INTO MeterData
                                 SELECT * 
                                 FROM DummyData 
                                 WHERE NOT EXISTS 
                                 (SELECT * 
                                  FROM MeterData 
                                  WHERE MeterData.MeterNumber != DummyData.MeterNumber 
                                  AND MeasurementDate = MeterData.MeasurementDate 
                                  AND MeasurementTime = MeterData.MeasurementTime
                                 )
                                 ORDER BY MeterNumber ASC;""")   
                   
                    c.execute("""UPDATE MeterData
                                 SET
                                 Consumption =  DummyData.Consumption, 
                                 FileName =  DummyData.FileName 
                                 FROM DummyData
                                 WHERE
                                 EXISTS (
                                         SELECT *
                                         FROM DummyData
                                         WHERE MeterNumber = MeterData.MeterNumber AND MeasurementDate = MeterData.MeasurementDate AND MeasurementTime = MeterData.MeasurementTime
                                        );""") 
        
                    c.execute("""DROP TABLE DummyData;""")
            

SQL Queries


In [7]:
c.execute('''SELECT * FROM FileSystem''').fetchall()

[('PN000001', 'SMRT', 'GAZ', 20191016, 102939),
 ('PN000002', 'SMRT', 'GAZ', 20191016, 102941),
 ('PN000003', 'SMRT', 'GAZ', 20191016, 102942),
 ('PN000004', 'SMRT', 'GAZ', 20191016, 102943),
 ('PN000005', 'SMRT', 'GAZ', 20191016, 102944),
 ('PN000006', 'SMRT', 'GAZ', 20191016, 102945),
 ('PN000007', 'SMRT', 'GAZ', 20191016, 102946),
 ('PN000008', 'SMRT', 'GAZ', 20191016, 102947),
 ('PN000009', 'SMRT', 'GAZ', 20191016, 102948),
 ('PN000010', 'SMRT', 'GAZ', 20191016, 102949)]

In [8]:
c.execute('''SELECT * FROM MeterData''').fetchall()    

[('000000002', 20191014, 1100, 11.48, 'PN000007'),
 ('000000002', 20191014, 1200, 11.48, 'PN000007'),
 ('000000002', 20191014, 1300, 11.48, 'PN000007'),
 ('000000002', 20191014, 1400, 11.48, 'PN000007'),
 ('000000002', 20191014, 1500, 11.48, 'PN000007'),
 ('000000002', 20191014, 1600, 11.48, 'PN000007'),
 ('000000002', 20191014, 1700, 11.48, 'PN000007'),
 ('000000002', 20191014, 1800, 11.48, 'PN000007'),
 ('000000002', 20191014, 1900, 11.48, 'PN000007'),
 ('000000002', 20191014, 2000, 11.48, 'PN000007'),
 ('000000002', 20191014, 2100, 11.48, 'PN000007'),
 ('000000002', 20191014, 2200, 11.48, 'PN000007'),
 ('000000002', 20191014, 2300, 11.48, 'PN000007'),
 ('000000002', 20191015, 0, 11.48, 'PN000007'),
 ('000000002', 20191015, 100, 11.48, 'PN000007'),
 ('000000002', 20191015, 200, 11.48, 'PN000007'),
 ('000000002', 20191015, 300, 1.13, 'PN000008'),
 ('000000002', 20191015, 400, 1.13, 'PN000008'),
 ('000000002', 20191015, 500, 1.13, 'PN000008'),
 ('000000002', 20191015, 600, 1.13, 'PN000

In [9]:
c.execute('''SELECT FileName FROM FileSystem''').fetchall()

[('PN000001',),
 ('PN000002',),
 ('PN000003',),
 ('PN000004',),
 ('PN000005',),
 ('PN000006',),
 ('PN000007',),
 ('PN000008',),
 ('PN000009',),
 ('PN000010',)]

In [10]:
c.execute('''SELECT * FROM MeterData WHERE MeterNumber='000000001' ''').fetchall()

[('000000001', 20191015, 600, 11.48, 'PN000007'),
 ('000000001', 20191015, 700, 11.48, 'PN000007'),
 ('000000001', 20191015, 800, 11.48, 'PN000007'),
 ('000000001', 20191015, 900, 11.48, 'PN000007'),
 ('000000001', 20191015, 1000, 11.48, 'PN000007'),
 ('000000001', 20191015, 1100, 11.48, 'PN000007'),
 ('000000001', 20191015, 1200, 12.07, 'PN000010'),
 ('000000001', 20191015, 1300, 12.07, 'PN000010'),
 ('000000001', 20191015, 1400, 12.07, 'PN000010'),
 ('000000001', 20191015, 1500, 12.07, 'PN000010'),
 ('000000001', 20191015, 1600, 12.07, 'PN000010'),
 ('000000001', 20191015, 1700, 12.07, 'PN000010'),
 ('000000001', 20191015, 1800, 12.07, 'PN000010'),
 ('000000001', 20191015, 1900, 12.07, 'PN000010'),
 ('000000001', 20191015, 2000, 12.07, 'PN000010'),
 ('000000001', 20191015, 2100, 12.07, 'PN000010'),
 ('000000001', 20191015, 2200, 12.07, 'PN000010'),
 ('000000001', 20191015, 2300, 12.07, 'PN000010'),
 ('000000001', 20191016, 0, 12.07, 'PN000010'),
 ('000000001', 20191016, 100, 12.07, '

In [11]:
c.execute('''SELECT count(*) FROM FileSystem''').fetchall()

[(10,)]

In [12]:
c.execute('''SELECT * FROM FileSystem WHERE (DateCreated,TimeCreated) IN (SELECT MAX(DateCreated),MAX(TimeCreated) 
from FileSystem)''').fetchall()

[('PN000010', 'SMRT', 'GAZ', 20191016, 102949)]